antes de ejecutar este script hay que compilar, por separado, los archivos java con el siguiente comando:
> javac -cp ecj-27.jar:. *.java

Esto generará los .class necesarios en el mismo directorio.

para ejecutar este sript usamos:
> jupyter nbconvert --to notebook --execute --inplace ejecuciones_y_guardado_datos.ipynb


código para ejecutar ECJ con las 9 configuraciones

In [ ]:
import subprocess
import time
from codecarbon import EmissionsTracker
from itertools import product
import re
import socket
COMPUTER_NAME = socket.gethostname()
codecarbon_file = f'codecarbon_ECJ_{COMPUTER_NAME}.csv'

# Parámetros a recorrer
population_sizes = [2**6, 2**10, 2**14]
crossover_probs = [0.2, 0.01, 0.8]

# Ruta al archivo de parámetros
params_file = "onemax.params"

# Leer el contenido base del archivo
with open(params_file, "r") as f:
    base_params = f.read()

# Función para actualizar el archivo con nuevos valores
def actualizar_params(pop_size, crossover_prob):
    nuevo = re.sub(r"pop\.subpop\.0\.size\s*=.*", f"pop.subpop.0.size = {pop_size}", base_params)
    nuevo = re.sub(r"pop\.subpop\.0\.species\.pipe\.source\.0\.prob\s*=.*",
                   f"pop.subpop.0.species.pipe.source.0.prob = {crossover_prob}", nuevo)
    with open(params_file, "w") as f:
        f.write(nuevo)

# Función para compilar los archivos
def compilar_java():
    print("Compilando archivos Java...")
    subprocess.run(["javac", "-cp", "ecj-27.jar:.", "*.java"], check=True)

# Bucle principal
for pop_size, crossover_prob in product(population_sizes, crossover_probs):
    print("\n=======================================")
    print(f"CONFIGURACIÓN: Población={pop_size}, Cruce={crossover_prob}")
    print("=======================================")

    # Actualizar archivo params
    actualizar_params(pop_size, crossover_prob)

    # Compilar el código
    # compilar_java()

    # Ejecutar 10 veces con medición
    for i in range(1, 11):
        print(f"\nEsperando 5 segundos antes de la ejecución {i}...")
        time.sleep(5)

        print(f"Ejecutando iteración {i} con CodeCarbon...")

        tracker= EmissionsTracker(
                        project_name="OneMax_Experiment",  # Mismo proyecto para todas las ejecuciones
                        output_dir=".",                        
                        output_file=codecarbon_file,
                        log_level="critical",
                        save_to_file=True,
                        tracking_mode="process",
                        allow_multiple_runs=True
                    )
        tracker.start()

        subprocess.run(["java", "-cp", "ecj-27.jar:.", "RunOneMax"], check=True)

        tracker.stop()

        print(f"Iteración {i} completada.")

print("\nEXPERIMENTO COMPLETO")